In [ ]:
import numpy as np
from skimage.io import imread
from skimage.feature import greycomatrix, local_binary_pattern
from skimage.exposure import rescale_intensity
from matplotlib import pyplot as plt
from scipy import ndimage as ndi

In [ ]:
img = imread('/scratch/hoerl/auto_sir_dna_comp/sorting20211108/069251b48328f1f51e3ff8d23e6e6173_overview370_detail0.png')

In [ ]:
plt.imshow(img)
img.max(), img.min()
np.unique(img)

In [ ]:
img2 = ndi.gaussian_filter(img, 0.8)
img2 = rescale_intensity(img2, out_range='uint8').astype(np.uint8)
glcm = greycomatrix(img2, [15], [np.pi/2])
plt.imshow(glcm.squeeze()[:100,:100], cmap='Reds')

In [ ]:
lbp = local_binary_pattern(img, 16, 3, method='uniform')

plt.figure(figsize=(10,10))
plt.imshow(lbp)
# plt.hist(lbp.ravel(), density=True, bins=16+2)


np.unique(lbp).shape

## 15.11.2021: New sorting to JSON

In [ ]:
import glob

# collect good/bad/questionable sortings
# NB: ?ad/?ood for case insensitivity
bad_imgs = set(glob.glob('/data/cooperation_data/ArgyrisPapantonis-nuclear_architecture/Sorting data_all/**/?ad/*.png', recursive=True))
good_imgs = set(glob.glob('/data/cooperation_data/ArgyrisPapantonis-nuclear_architecture/Sorting data_all/**/?ood/*.png', recursive=True))
questionable_imgs = set(glob.glob('/data/cooperation_data/ArgyrisPapantonis-nuclear_architecture/Sorting data_all/Questionable/*.png'))

# check that we have all 500 imgs
len(bad_imgs | good_imgs | questionable_imgs)

In [ ]:
import os
import re
import json
from itertools import chain, repeat
from collections import defaultdict

res = defaultdict(list)

for classification, img_path in chain(zip(repeat('bad'), bad_imgs), zip(repeat('good'), good_imgs)):

    img_filename = os.path.split(img_path)[1]
    file, dataset = re.match('(.*?)_(overview.*?_detail.*?)\.png', img_filename).groups()
    res[classification].append((file, dataset))
    
# res

In [ ]:
with open('/scratch/hoerl/auto_sir_dna_comp/sorting20211115.json', 'w') as fd:
    json.dump(res, fd, indent=1)